# Shipping route optimization with D'wave quantum computing

# Introduction and about this code (Nick Stuke)

Quantum computers have been in development for quite a while. They are intended to solve various problems that scale too violently even for supercomputers. Thus, D'Wave has already started its development since 1999 on quantum computers.\[3\] The aim of this Jupyter notebook is to check the current state of the art for feasibility. For this we will try to solve an example on the quantum computers and processes provided by D'Wave. In the following, we will therefore use the D'Wave software libraries, which are intended to enable efficient use with quantum computers.

The example is a shipping route optimization. A CSV list is given as input, in which the connections of the various ports are listed. The goal is to find the shortest route from a starting port to the ending port. The list is in the data_harbours_utf8.csv dataset. An output of the route is therefore expected.

We will try to solve this example using D'Wave's methods. We use two different methods for this: Constrained Statisfaction Problem(CSP) and Constrained Quadratic Model(CQM). With these two methods, we have the option of either providing direct input to the quantum computer or using D'Wave's hybrid process.
In addition, we will look for the optimal solution with classic algorithms in order to have a validation of the result.

## Remark

This code was written in such a way that the best possible understanding can be conveyed. The goal wasn't to write good code for production purposes. Therefore, redundant parts and further gaps are possible and present.

When executing the code, the environment from the venv folder must be used. This is because code changes were made to the packages to make them work.

# Packages-Import Area

In order to use the code the following libraries, classes and functions must be installed:

In [29]:
from __future__ import print_function
import pandas as pd
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system import LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
from dimod import quicksum
from dimod import Binary
from dimod import CQM
import dwave.inspector
import dwavebinarycsp
import re
import multiprocessing
import time
import pprint
import sys
import threading
from time import sleep
import _thread as thread

In [ ]:
# Use static Departure and Destination?
static_departDesti=True

# Input and Preprocessing (Anna Ehrenberg)

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

In this section the following input is proceeded:

Input data file with information on ferry routes with
- From: route start port (Column1)
- To: route end port (Row1)
- Distance: shortest direct distance between departure and arrival port (Number in matrix). If the connection is not possible, the distance is zero or no value is entered.

Both a large and a small data set are read in as input.


### Input data file

In [ ]:
df_ports = pd.read_csv("data_harbours_utf8.csv", sep = ";")
df_ports_sample = pd.read_csv("sample_data_harbours_utf8.csv", sep = ";")
df_ports

### Formatting of data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured into an unstacked format. The new table contains of the follwoing columns:
1. From
2. To
3. Distance
After tranforming the format, only possible routes are included in the data.

In [ ]:
def format_ports(ports):
    # rename 'Column1' to 'From'
    return ports.rename(columns={'Column1':"From"})
df_ports    	= format_ports(df_ports)
df_ports_sample = format_ports(df_ports_sample)

def create_routes(ports):
    # transform end harbours in columns to rows
    routes= pd.melt(ports,id_vars=['From'],var_name="To",value_name='Distance')

    # filter NaN-values
    routes=routes[routes["Distance"].notnull()]

    # filter out 0.0 values
    routes=routes[routes.Distance != 0]

    #reset index
    routes=routes.reset_index(drop=True)
    return routes
df_routes        = create_routes(df_ports)
df_routes_sample = create_routes(df_ports_sample)

df_routes_withoutStardEnd=df_routes
df_routes_withoutStardEnd_sample=df_routes_sample
df_routes

### User input and validation of departure and destination harbour 

In this section the manual user input is requested and validated by the following criteria:
1. departure and destination ports exist in the input file with connections
2. departure port is not equal to destination port

If input is invalid, the according error message is displayed.

In [ ]:
if static_departDesti:
    departure="Brunsbüttel"
    destination="Emden"
else:
    #Check whether the departure port is element of the input data
    while True:
        #Entering departure port
        departure=input(str("Please enter departure port:"))
        if not departure in df_routes['From'].values:
            print("Entered port is not defined as a depature port in the input file. Please enter valid departure port.")
            continue
        else:
            break

    #Check whether the destination port is element of the input data and not equal to departure port
    while True:
        #Entering destination port
        destination=input(str("Please enter destination port:"))
        if not destination in df_routes['To'].values:
            print("Entered port is not defined as a destination port in the input file. Please enter destination valid port.")
            continue
        if destination == departure:
            print("Entered destination port is equal to departure port. Please enter valid destination port.")
            continue
        else:
            #we're happy with the value given.
            break


# Data Preparation for Quantum Computing (Anna Ehrenberg, Nick Stuke)

In this section, all methods are defined that are later used to
1. create Constraint Satisfaction Problem (CSP)
2. create Constraint Quadratic Model (CQM)
3. formulate Binary Quadratic Model (BQM) as QUBO. 
4. add Penalty to the BQM

The basic structure of the CSP and CQM is formulated in the Ferryarea class. How the formulation of this works is then explained below for the respective function calls. The CSP is then reformulated into the BQM. In this example, the problem is formulated as a BQM in the form of a QUBO. Finally, we modify the BQM so that longer distances are penalized.

### Basic functions for CSP

In order to simplify code in the constraints section, the following functions are defined. 

In [ ]:
def sum_to_two_or_zero(*args):
        """Checks to see if the args sum to either 0 or 2.
        """
        sum_value = sum(args)
        return sum_value in [0, 2]
        
def sum_smaller_equal_one(*args):
        """Checks to see if the args sum to either smaller or equal to 1.
        """
        sum_value = sum(args)
        return sum_value in [0, 1]

def get_labels(dataframe):
        """Returns a list of labels from a Dataframe of the format of the input file"""
        labels=(dataframe['From']+'-'+dataframe['To']+'-'+dataframe['Distance'].astype(str)).values.tolist()
        return labels

### Definition class Ferryarea

In [ ]:
class Ferryarea:
    def __init__(self,start,end,routes):
        """ Initializes object of Type Ferryarea"""
        #Instantiate
        self.start=start
        self.end=end
        self.df_routes = routes
        self.ports = pd.DataFrame({'ports':np.append(self.df_routes['To'].unique(),self.df_routes['From'].unique())}).drop_duplicates().values.tolist()
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        self.cqm = CQM()
        self.objectiveDistances=0
        #Add start and end route
        self.df_routes= self.df_routes.append({'From':self.end, 'To':'end','Distance':0}, ignore_index=True)
        self.df_routes= self.df_routes.append({'From':'start', 'To':self.start,'Distance':0}, ignore_index=True)
        
    def _get_csp(self):
        return self.csp
    def _get_cqm(self):
        return self.cqm    
    def _apply_valid_routes_constraint(self):
        """Creates constraints defining that every entered port has to be exited as well and that every port can only be entered and exited through one route."""

        for port in self.ports:
            #assign to directions Dataframe a Dataframe containing only direction to and from the port
            df_directions_from= self.df_routes[self.df_routes['From']==port[0]]
            df_directions_to= self.df_routes[self.df_routes['To']==port[0]]
            df_directions= pd.concat([df_directions_to,df_directions_from])
            
            #creates sets of labels as basis for constraint defintion
            directions_all = set(get_labels(df_directions))
            
            directions_to= set(get_labels(df_directions_to))
            
            directions_from= set(get_labels(df_directions_from))
              
            x={r: Binary(
                "%r" %r
            )for r in directions_all}

            for d in directions_all:
                distance=float(d.split('-')[2])
                self.objectiveDistances=self.objectiveDistances+distance*x[d]
            
            self.cqm.add_constraint(
                quicksum([x[tod] for tod in directions_to])
                -
                quicksum([x[fromd] for fromd in directions_from]) 
                == 0)
            
            #add constraint sum_to_two_or_zero 
            self.csp.add_constraint(sum_to_two_or_zero,directions_all)

            #add constraint 'To' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one,directions_to)
            
            #add constraint 'From' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one, directions_from)

    def _set_CQM_objective(self):
        self.cqm.set_objective(self.objectiveDistances)
        
    def _set_start_and_end(self):
        """Sets the values of the departure and destination port to 1 and all other start and end possibilities to 0
        """ #all other possibilities do not have to be 0. However, this is expected to descrease calculation time

        #select all start and end routes
        df_routes_to_fix= pd.concat(
            [self.df_routes[self.df_routes['From']=='start'],self.df_routes[self.df_routes['To']=='end']]
            )
        count=0

        for i in df_routes_to_fix.values:
            label= get_labels(df_routes_to_fix.iloc[[count]])
            if (df_routes_to_fix['To'].iloc[count])==self.start:
                #sets departure to 1
                self.csp.fix_variable(label[0],1)
            elif (df_routes_to_fix['From'].iloc[count])==self.end:
                #sets destination to 1
                self.csp.fix_variable(label[0],1)
            else:
                #sets departure and destination to 0
                self.csp.fix_variable(label[0],0)
            count=count+1

        self.cqm.fix_variable("'start-"+self.start+"-0.0'",1)
        self.cqm.fix_variable("'"+self.end+"-end-0.0'",1)
       

First we initialize the class "Ferry Area". For this, the possible routes and the start and end port are transferred.
Both the CSP and the CQM require constraints with variables to return a solution. Since we want to find out the shortest route, these are our variables, each encoded in binary. If a route is to be used, the variable has the value (1) and if not used, the value (0). Choosing the ports as variables makes less sense, as this can create ambiguity.

If we want to tell the CQM or CSP which port is the departure or destination, we need to fix certain variables. Since we chose routes as variables, two more artificial routes are added during initiation. One route to the starting port with distance 0 and one away from the ending port. We can fix these variables later on.

In [ ]:
f=Ferryarea(start=departure,end=destination,routes=df_routes)

The constraints are now set below.

For the CSP, two constraints ensure, that the problem is displayed correctly:
1. All routes connected to one port have to sum up to 0 or 2. This ensures that every port that is entered is exited as well.
2. The routes are one-way. This means it is possible to go from one port to another but not back the same way. With the first constraint is would be possible to take two routes where the port is listed in the "from" column. This means, it would be exited twice but never approached. Thus, another constraint is needed to ensure that ports are approached only once and existed only once. 

For the CQM we only need one constraint. In addition, we can already carry out preliminary work on the optimization function:
1. The following must apply to each port: (The sum of all outgoing routes)-(The sum of all incoming routes) must be equal to 0. This ensures that there are always the same number of incoming and outgoing routes. So whether the port is approached (both 1) or not (both 0). Larger numbers can later be excluded by the optimization function, since otherwise the route would be longer.
2. Since each port is already iterated over, we also build the optimization function. As a result we get the sum of all distances of the routes depending on their binary value. Given two routes with distance 12 and 15, we would have the following function: f=route1*12+route2*15 where route1 and route2 can take the value 0 or 1 depending on whether they are used.


In [ ]:
f._apply_valid_routes_constraint()

After all constraints are applied, start and end routes are fixed to 1. 

In [ ]:
f._set_start_and_end()

In addition, the objective for the CQM is defined.

In [ ]:
f._set_CQM_objective()

Now the two defined models are called from the class

In [ ]:
csp= f._get_csp()
cqm=f._get_cqm()

While the CQM can be processed directly by the hybrid process, CSP must first be transformed into a BQM. In order to fit the Problem to the modell it is necessary to adjust the following parameters:
- max_graph_size: This parameters limits the number of variables that can be included in one constraint. In this case, the number of connections to and from one port. The original d-wave function dwavebinarycsp.stitch uses the function np.unpackbits from the numpy library. This only unpacks arrays from the size of 8 bits. Thus, the function all_possible() and the dtype of the matrix A in the file generation.py have been modified. Now, up to 32 connections between variables can be entered. 

The following code also has a time limit because D'Wave's CSP function takes a long time depending on the amount of input. Processing stops after 5 minutes.

In [ ]:
def quit_function(fn_name):
    sys.stderr.flush() 
    thread.interrupt_main()
def exit_after(s):
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer


def stich_function(inputCsp, graph_size):
    timeout=False
    bqm=None

    @exit_after(120)
    def runStich(csp, graph_size):
        stitchReturn = dwavebinarycsp.stitch(csp,max_graph_size=graph_size)
        return stitchReturn

    try:
        bqm = runStich(inputCsp,graph_size)
    except KeyboardInterrupt:
        timeout=True
        print("Stich-Function runs too loo long")
    return timeout, bqm


csp_stitchTimeout, bqm = stich_function(csp,12)
    

### Results BQM

The BQM is created by the stitch() function provided by d-wave. However, the larger the input data gets the longer is takes for the stitch() function to built the BQM. Within the function a graph is built with all variables. Using the sample data, a graph of 9 nodes and 36 edges is created and the BQM can be built in approximately 42 seconds. For a graph with 10 nodes, approximiately 15 minutes are needed.
In conclusion, it is not practical to use the BQM model for real world problems. 

So far, the model can find possible ways between two ports. In the following code, a penalty is added in order to penalize long distances. This way, the shortest route is selected. However, the number of stops is not taken into consideration. 

In [ ]:
def addPenalty(inputBqm, csp_stitchTimeout):
    if not csp_stitchTimeout:
        # Add Penalty
        penalty=0.01
        for v in inputBqm.variables:
            # Ignore auxiliary variables
            if isinstance(v, str) and re.match(r'^aux\d+$', v):
                continue
            split_v = v.split('-')
            inputBqm.add_variable(v, penalty*float(split_v[2]))
    return inputBqm

if not csp_stitchTimeout:
    bqm=addPenalty(bqm,csp_stitchTimeout)

# Transfer to Quantum Sampler (Anna Ehrenberg, Nick Stuke)

In this section, the before built models are tranferred to the quantum computer by using D-wave's Samplers.

## BQM-Sampling and Result-Selection (Anna Ehrenberg, Nick Stuke)

The sampler for BQMs is assigned.

In [ ]:
# define Sampler
sampler = EmbeddingComposite(DWaveSampler())

The sampler is called with the input BQM, number of reads and chain strengths. All results are saved with the produced energy. The result with the lowest energy is expected to be the best result. 

The "num_reads" and "chain_strength" parameters were set according to a standard.

The "Chain_strength" must be fixed depending on the input value. A general guide is the highest absolute value. The chain strength should be set just above. The chain strength is there when the problem graph cannot be mapped onto the qubits of the quantum computer, so that two qubits then assume the same state due to the highest possible negative value and are therefore considered one qubit. This is how new connections can be created.

With the "num_reads" the scattering of the energy histogram has to be looked at. If the energy values are close together, several processes have to be run through. Num_reads reflects the number of quantum annealing processes. The further apart different solutions with the energy level are, the easier it is for the quantum computer to assume the lowest energy state.

In [ ]:
def sample_bqm(inputBqm, sampler):
    cpuTime_start=time.process_time() 
    result = sampler.sample(inputBqm,
                        num_reads=100,
                        chain_strength=3,
                        label='ferrylines')
    cpuTime_end=time.process_time()
    cpuTime=cpuTime_end-cpuTime_start
    return result, cpuTime
calcTime_csp=0
if not csp_stitchTimeout:
    # run BQM on QPU
    result,calcTime_csp = sample_bqm(bqm,sampler)

The results can be visualized online with the D-wave inspector. This is not saved in the notebook, but must be executed in order for this to be visible.

In [ ]:
if not csp_stitchTimeout:
    # visualize result
    dwave.inspector.show(result)

The final result is displayed.

In [ ]:
if not csp_stitchTimeout:
    print(result.first.sample)

## CQM-Sampling and Result-Selection (Nick Stuke)




Now the CQM is running. Since it runs through the hybrid process at D'Wave, no further parameters need to be specified here. The following code only measures the processing time, how long the process takes.

In [ ]:
sampler_cqm = LeapHybridCQMSampler()
def sample_cqm(cqm, sampler):
    cpuTime_start=time.process_time() 
    sample_set=sampler_cqm.sample_cqm(cqm)
    cpuTime_end=time.process_time()
    cpuTime=cpuTime_end-cpuTime_start
    return sample_set, cpuTime
sample_set_cqm, calcTime_cqm=sample_cqm(cqm,sampler_cqm)

Then the best solution is found. If no valid solution can be found with the set constraints, an exception is thrown.

In [ ]:
n_samples = len(sample_set_cqm.record)
feasible_samples = sample_set_cqm.filter(lambda d: d.is_feasible) 
best_feasible_cqm_solution=DWaveSampler.sample
if not feasible_samples: 
    raise Exception("No feasible solution could be found for this problem instance.")
else:
    best_feasible_cqm_solution = feasible_samples.first
    print(best_feasible_cqm_solution)
best_feasible_cqm_solution

# Shortest Way with classical Computing (Nick Stuke)

This Chapter show an algorithm on a classical computer with no quantum-computer ressources. A common algorithm for the given problem is the Dijkstra-Algorthm\[1\](vgl. S.1).
As a Greedy-Algorithm Dijkstra processes each node once, so with more nodes, it needs more process-time.
A* is a much faster algorithm but don't find always the optimal solution.\[2\](vgl. S.2)
Since the quantum computer should have exactly the strengths of quickly finding global minima, the Dijkstra algorithm is used in this notebook, so that we have a correct result as a control.

## Define Graph
First of all, we have to create a graph. Therefore we iterate over every port and add the routes away from it to the graph-note. The graph is stored as as dictonary.

In [ ]:
# define Graph with distances for every Point
def defineGraph(ports,routes):
    graph={}
    for index,port in ports.iterrows():
        port_routes=routes[routes['From']==port['From']]
        distances={}
        for i,r in port_routes.iterrows():
            distances[r['To']] = r['Distance']
        graph[port['From']] =distances
    print(graph)
    return graph
graph=defineGraph(df_ports,df_routes_withoutStardEnd)


## Calculate shortest distance to start-port
The code below contains the dijkstra alogrithm. For each port, it is checked how far the shortest distance to the starting port is and which port is next on the shortest way to the starting port. 


In [ ]:
def calcShortestDistances(depart,graph):
    queue = [depart]
    d = {node: {"shortest distance":float("inf"), "previous":None} for node in graph}
    d[depart]["shortest distance"] = 0
    cpuTime_start=time.process_time() 
    while queue:
        current = queue.pop(0)
        shortest_distance = d[current]["shortest distance"]
        for neighbour in graph[current]:

            dist_to_neighbour = graph[current][neighbour]

            if shortest_distance + dist_to_neighbour < d[neighbour]["shortest distance"]:

                d[neighbour] = {
                    "shortest distance": shortest_distance + dist_to_neighbour,
                    "previous": current
                }
                queue.append(neighbour)
    cpuTime_end=time.process_time()
    cpuTime=cpuTime_end-cpuTime_start
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(d)
    print() 
    return d,cpuTime

d, calcTime_dji=calcShortestDistances(departure,graph)

## Create Path with shortest distances

The previous code now determined the shortest route to the starting port for all ports. In the following, the route is now defined based on these determined values. 
Starting with the port of destination, the next port to the port of departure is determined. Every Port is packed in a list, which reflects the optimal route.


In [ ]:
def buildPath(d,dest,depart):
    current = dest
    print('Destination: '+str(dest))
    print('Departure: '+str(depart))
    dji_path = [current]
    distance_sum=0.0
    while current != depart:
        print(current)
        current = d[current]["previous"]
        dji_path.append(current)

    dji_path = dji_path[::-1]
    print('Distance: '+str(d[destination]["shortest distance"])+' Pfad: '+str(dji_path))
    return dji_path

dji_path=buildPath(d,destination,departure)

# Result-Consideration (Nick Stuke)

## Additional Result-Production

In the code below, more results are produced to allow comparison of processing. A smaller amount of data is used for this. The aim is to identify possible changes in processing based on different amounts of data.
The same code as before is used, so no further explanation is given.

In [ ]:
departure_sample="Bremerhaven"
destination_sample="Brunsbüttel"
f_sample=Ferryarea(start=departure_sample,end=destination_sample,routes=df_routes_sample)
f_sample._apply_valid_routes_constraint()
f_sample._set_start_and_end()
f_sample._set_CQM_objective()
csp_sample= f_sample._get_csp()
cqm_sample=f_sample._get_cqm()
csp_stitchTimeout_sample, bqm_sample = stich_function(csp_sample,10)
if not csp_stitchTimeout_sample:

    bqm_sample=addPenalty(bqm_sample,csp_stitchTimeout_sample)
calcTime_csp_sample=0

if not csp_stitchTimeout_sample:
    # run BQM on QPU
    result_sample,calcTime_csp_sample = sample_bqm(bqm_sample,sampler)
sample_set_cqm_sample, calcTime_cqm_sample=sample_cqm(cqm_sample,sampler_cqm)
n_samples_sample = len(sample_set_cqm_sample.record)
feasible_samples_sample = sample_set_cqm_sample.filter(lambda d: d.is_feasible) 
best_feasible_cqm_solution_sample=DWaveSampler.sample
if not feasible_samples_sample: 
    raise Exception("No feasible solution could be found for this problem instance.")
else:
    best_feasible_cqm_solution_sample = feasible_samples_sample.first
    print(best_feasible_cqm_solution_sample)
graph_sample=defineGraph(df_ports_sample,df_routes_withoutStardEnd_sample)
d_sample, calcTime_dji_sample=calcShortestDistances(departure_sample,graph_sample)
dji_path_sample=buildPath(d_sample,destination_sample,departure_sample)


## Result-Homogenization

In the following area, a brief processing of the results is carried out. This is required for easier presentation and does not change or add any information.

In [ ]:
result_dji=dji_path
result_dji_sample=dji_path_sample

def buildSolutionPath(solutionSet,depart,desti):
    result=[depart]
    solution=[]
    for s in solutionSet.sample:
        if (solutionSet.sample[s]==float(1) and 
         s.find('end')==-1 and 
         s.find('start')==-1):
            solution.append(s)
    currentP=depart
    while currentP!=desti:
        for s in solution:
            s=s.replace("'","").split('-')
            if s[0].find(currentP)==0:
                currentP=s[1]
                result.append(s[1])
                break
    return result
result_cqm=buildSolutionPath(best_feasible_cqm_solution,departure,destination)
result_cqm_sample=buildSolutionPath(best_feasible_cqm_solution_sample,departure_sample,destination_sample)
result_csp=['Timeout']
if not csp_stitchTimeout:
    result_csp=buildSolutionPath(result.first,departure,destination)
result_csp_sample=['Timeout']
if not csp_stitchTimeout_sample:
    result_csp_sample=buildSolutionPath(result_sample.first,departure_sample,destination_sample)


## Result-Summary

In the following, the results are presented in the form of a route from the start to the end destination. Which algorithm each belongs to is included in the Python output.


In [ ]:
print('Result-CQM: '+str(result_cqm))
print('Result-CSP: '+str(result_csp))
print('Result-Dijkstra: '+str(result_dji))
print('Result-CQM_Sample: '+str(result_cqm_sample))
print('Result-CSP_sample: '+str(result_csp_sample))
print('Result-Dijkstra_Sample: '+str(result_dji_sample))


The following code contains the required processing times in milliseconds for the different variants. The times only include the search for the solution. Times for problem formulation are not included.

In [ ]:
print('Process Time for CQM: '+str(calcTime_cqm))
print('Process Time for CSP: '+str(calcTime_csp))
print('Process Time for Dijkstra: '+str(calcTime_dji))
print('Process Time for CQM-Sample: '+str(calcTime_cqm_sample))
print('Process Time for CSP-Sample: '+str(calcTime_csp_sample))
print('Process Time for Dijkstra-Sample: '+str(calcTime_dji_sample))


On a small set of data, all three algorithms were able to find the correct solution. Djikstra is considered a comparison here, since the solution is calculated individually and is therefore seen as correct. The processing times are not relevant because they are too short.

The situation is different with a larger data set. With the CSP, the stitch function took too long and was therefore aborted. The reason for the long execution time has already been explained in the execution of the stitch-function. The other two algorithms were able to calculate the route with negligible time.In order to see a difference in time, a significantly larger data set would have to be read.

# Conclusion (Nick Stuke)

As can be seen, quantum computing can already be created with problems. With the small example of the ferry route optimization, the desired solution could be determined with the CQM model using quantum computing. A solution could also be found with the CSP for the small data set, but with the large data set you can see that it leads to scaling problems and runtimes that are too long. Thus, there is a barrier in creating the optimization function in the case of the CSP. More research would have to be done here in the direction of creating an optimization function.
In order for the statement as to whether quantum computing can already be used in practice to be answered, one would also have to work with larger data and with exponentially scaling problems.

So, yes, you can solve problems today with quantum computers. To what extent and what limitations there are still needs to be worked out further.

# Quellen

\[1\] Broumi, Said, Assia Bakal, Mohamed Talea, Florentin Smarandache, und Luige Vladareanu. „Applying Dijkstra algorithm for solving neutrosophic shortest path problem“. In 2016 International Conference on Advanced Mechatronic Systems (ICAMechS), 412–16. Melbourne, Australia: IEEE, 2016. https://doi.org/10.1109/ICAMechS.2016.7813483.

\[2\] Candra, Ade, Mohammad Andri Budiman, und Kevin Hartanto. „Dijkstra’s and A-Star in Finding the Shortest Path: a Tutorial“. In 2020 International Conference on Data Science, Artificial Intelligence, and Business Analytics (DATABIA), 28–32. Medan, Indonesia: IEEE, 2020. https://doi.org/10.1109/DATABIA50434.2020.9190342.

\[3\] „Welcome to D-Wave — D-Wave System Documentation documentation“. Zugegriffen 13. Januar 2023. https://docs.dwavesys.com/docs/latest/c_gs_1.html.
